In [1]:
base_prompt= """You are a lead software engineer. You have been tasked with developing a {change_type}. 
You will be provided with a BDD requirement as well as a set of relevant code snippets and the codebase structure to support your decision making. 
Your job is to review the requirement and the code snippets and then generate a set of steps to implement the requirement.

Your guidelines:
- If you are uncertain about whether to edit an exisitng bit of code or generate new one, you should err on the side of generating new code.
- You should be specific in your instructions, detailing the variable names/types, method names/return types, etc.
- The only actions that can be taken on your instructions are to add new code or edit existing code. No other actions are allowed.
- You may not ask for more information about the task. You must complete the task with the information provided.
- Each step should only refer to a coding instruction. DO NOT add steps for code reviews or any non-coding tasks.

IMPORTANT: You should return a list of steps to implement the requirement.
Each step should contain the following fields:
    - step_number: The step number (to be carried out in order to avoid compilation errors)
    - instruction: A description of what the user carrying out the task should do, be detailed.
    - explanation: A explanation of why the step is necessary, why it is independent of the other steps and why it is a correctly sized task.
    
Example steps you should return:
    [
        {{
            "step_number": 1,
            "instruction": "Add a price field to the product model. The type should be a double and the default value should be 0.0",
            "explanation": "This step is necessary to store the price of the product. It is independent of the other steps because it does not rely on any other code changes. It is correctly sized because it is a single action that can be taken by a coder."
        }},
        {{
            "step_number": 2,
            "instruction": "Add a query to the product repository to find all products with a price greater than 100.0.",
            "explanation": "This step is necessary to allow the system to query products based on price. It is independent of the other steps because it does not rely on any other code changes. It is correctly sized because it is a single action that can be taken by a coder."
        }},
    ...
        {{
            "step_number": 5,
            "instruction": "Add a controller to the codebase that will call the service method you just created.",
            "explanation": "This step is necessary to allow the system to interact with the new functionality. It is independent of the other steps because it does not rely on any other code changes. It is correctly sized because it is a single action that can be taken by a coder."
        }}
    ]

Only return json and only return ordered steps in the above format.

Here is the BDD requirement:

"{user_request}"

Here is the repo structure and relevant code snippets:

"{repo_tree}"
"""

In [2]:
from uuid import uuid4
import dotenv
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from typing import List
import json

dotenv.load_dotenv()

# class SuggestedUpdate(BaseModel):
#     original: str = Field(..., description="The original value of the section of the prompt should be updated")
#     updated: str = Field(..., description="The updated value of the section of the prompt should be updated")
    
class ApprovalResponse(BaseModel):
    original_prompt: str = Field(..., description="The original prompt that was reviewed")
    approved: bool = Field(..., description="Whether the request was approved or not, True or False")
    explanation: str = Field(..., description="A detailed explanation of why the request was approved or not")
    # partial_updates: List[SuggestedUpdate] = Field(..., description="The sections of the prompt need to be updated, and the new values")

class PromptReviewPrincipals:
    def __init__(self, core_principles: List[str]):
        self.core_principles = core_principles
    
    def add_principle(self, principle: str):
        """
        Adds a principle to the core principles list.
        
        :param principle: The principle to be added.
        """
        self.core_principles.append(principle)
        
    def __str__(self):
        """
        Returns a string representation of the core principles, each principle is listed on a new line with a preceding dash.
        
        Example:
        - principle 1
        - principle 2
        ...
        """
        return "\n".join([f"- {principle}" for principle in self.core_principles])
    
class PromptReviewAgent:
    """
    This class represents an approval agent that reviews and approves requests based on its persona and core principles.
    """
    def __init__(self, persona_title: str, core_principles: PromptReviewPrincipals, model:str='gpt-4o', temperature: float=1.0):
        """
        Initializes the PromptReviewAgent with a persona title, core principles, model, and temperature.
        
        :param persona_title: The title of the persona for the agent.
        :param core_principles: The core principles of the agent.
        :param model: The model to be used by the agent. Default is 'gpt-4o'.
        :param temperature: The temperature to be used by the agent. Default is 0.0.
        """
        self.id = uuid4()
        self.persona_title = persona_title
        self.core_principles = str(core_principles)
        self.llm = ChatOpenAI(temperature=temperature,model=model)
    
    async def review(self, prompt_to_review: str) -> ApprovalResponse:
        """
        Reviews a given prompt and returns an ApprovalResponse.
        
        :param prompt_to_review: The prompt to be reviewed.
        :return: An ApprovalResponse object containing the review results.
        """
        prompt_template = """You are a senior {persona_title}. You have been asked to review the following request and provide your approval or disapproval. 
        You should consider the request in light of your core principles and provide a detailed explanation on why you approve or disapprove of the request.
        Be constructive in your feedback and provide suggestions on how the request could be improved.
        
        **Your core principles are:**
        {core_principles}
        
        **Strict guidelines:**
        1. All restrictions already stated in the text being reviewed should not be modified.
        2. All negations should be preserved. e.g. "DO NOT" should not be changed to "DO" or removed entirely.
        3. All placeholders denoted by curly braces should not be modified or removed.
        4. Adding additional restrictions or negations is allowed.
        5. Adding additional placeholders is not allowed.
        6. Adding additional instructions to achieve your core principles is allowed.
        
        **Request to review:**
        -------------------------------------
        {prompt_to_review}
        -------------------------------------
        
        {format_instructions}
        """
        pydantic_parser = PydanticOutputParser(pydantic_object=ApprovalResponse)
        format_instructions = pydantic_parser.get_format_instructions()

        prompt = ChatPromptTemplate.from_template(prompt_template)
        messages = prompt.format_messages(persona_title=self.persona_title, prompt_to_review=prompt_to_review,core_principles=self.core_principles, format_instructions=format_instructions)

        for _ in range(3):
            try:
                output = self.llm(messages=messages)
                approval_summary = pydantic_parser.parse(output.content)
                break
            except Exception:
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")
        return approval_summary

class PromptAgent:
    """
    This class represents a prompt agent that manages the base prompt and updates it based on suggested updates.
    """
    def __init__(self, base_prompt: str):
        """
        Initializes the PromptAgent with a base prompt.
        
        :param base_prompt: The base prompt to be managed by the agent.
        """
        self.base_prompt = base_prompt
        self.llm = ChatOpenAI(model='gpt-4o', temperature=0.5)

        
    # async def update(self, suggested_update: SuggestedUpdate)->str:
    #     """
    #     Updates the base prompt based on a given SuggestedUpdate and returns the updated prompt.
        
    #     :param suggested_update: The SuggestedUpdate to be applied to the base prompt.
    #     :return: The updated base prompt.
    #     """
    #     for update in suggested_update:
    #         self.base_prompt = self.base_prompt.replace(update.original, update.updated)
            
    #     return self.base_prompt

    async def update(self, orgiginal_prompt: str, explanation: str)->str:
        """
        Updates the base prompt based on explanantion given in the ApprovalResponse and generates an improved prompt.
        """

        prompt_template = """You are an expert prompt engineer. You have been tasked with updating the following prompt based on the feedback provided by a reviewer.
        You should consider the feedback provided and update the prompt accordingly.

        **Original Prompt:**
        {original_prompt}

        **Feedback:**
        {explanation}

        Output the updated prompt.
        """
        prompt = ChatPromptTemplate.from_template(prompt_template)
        messages = prompt.format_messages(original_prompt=orgiginal_prompt, explanation=explanation)

        try:
            output = self.llm(messages=messages)
            self.base_prompt = output.content
        except:
            return self.base_prompt
                
        return self.base_prompt
        
        

In [3]:
class SequentialPromptReview:
    """
    This class represents a sequential approval process. It registers agents and manages the approval process.
    """
    def __init__(self, prompt_agent: PromptAgent, max_resets: int = 10):
        """
        Initializes the SequentialPromptReview with a prompt agent and a maximum number of resets.
        
        :param prompt_agent: The agent that provides the prompt for the approval process.
        :param max_resets: The maximum number attempts to reach approval consensus.
        """
        self.approvers = []
        self.prompt = prompt_agent.base_prompt
        self.base_prompt = prompt_agent.base_prompt
        self.prompt_agent = prompt_agent
        self.approved = {}
        self.max_resets = max_resets

    def register_approver(self, approver: PromptReviewAgent):
        """
        Registers an approver for the sequential approval process.
        
        :param approver: The approver to be registered.
        """
        self.approvers.append(approver)

    def reset_approval(self):
        """
        Resets the approval process by clearing the approved dictionary.
        """
        self.approved = {}

    async def review(self) -> str:
        """
        Executes the approval process. It iterates over the registered approvers and asks them to review the prompt.
        If an approver does not approve, the prompt is updated and the approval process is reset.
        The process continues until all approvers have approved or the maximum number of resets has been reached.
        
        :return: The final approved prompt.
        """
        reset_count = 0
        while len(self.approved) < len(self.approvers) and reset_count < self.max_resets:
            for approver in self.approvers:
                if approver.id not in self.approved:
                    result = await approver.review(self.prompt)
                    if result.approved:
                        self.approved[approver.id] = result
                    else:
                        self.prompt = await self.prompt_agent.update(result.original_prompt, result.explanation)
                        self.reset_approval()
                        reset_count += 1
                        break
                    
        if reset_count >= self.max_resets:
            return self.base_prompt
        
        return self.prompt

In [4]:
for _ in range(0):
    code_quality_principals = PromptReviewPrincipals([
        "Always write clean code",
        "Always write tests",
        "Always write documentation"
        ])
    code_quality_reviewer = PromptReviewAgent("software engineer", code_quality_principals)
    # result = await code_quality_approver.review("write a function that takes in a list of numbers and returns the sum of the numbers, and example output is sum([1,2,3]) = 6")
    result = await code_quality_reviewer.review(base_prompt)

    print(json.dumps(result.model_dump(), indent=4))

In [5]:
for _ in range(0):
    architecture_quality_approver = PromptReviewAgent("software architect", "1. Always design for scalability\n2. Always design for maintainability\n3. Always design for performance")
    # result = await code_quality_approver.generate("write a function that takes in a list of numbers and returns the sum of the numbers, and example output is sum([1,2,3]) = 6")
    result = await architecture_quality_approver.review(base_prompt)
    print(json.dumps(result.model_dump(), indent=4))

In [6]:
# prompt_agent = PromptAgent(base_prompt)
# code_quality_principals = PromptReviewPrincipals([
#     "Always write clean code",
#     "Always write tests",
#     "Always write documentation"
#     ])
# code_quality_reviewer = PromptReviewAgent("software engineer", code_quality_principals)

# architecture_quality_principals = PromptReviewPrincipals([
#     "Always design for scalability",
#     "Always design for maintainability",
#     "Always design for performance"
#     ])
# architecture_quality_reviewer = PromptReviewAgent("software architect", architecture_quality_principals)

# sequential_prompt_review = SequentialPromptReview(prompt_agent)
# sequential_prompt_review.register_approver(code_quality_reviewer)
# sequential_prompt_review.register_approver(architecture_quality_reviewer)
# final_prompt = await sequential_prompt_review.review()

# with open("prompt_before.txt", "w") as f:
#     f.write(base_prompt)

# with open("prompt_after.txt", "w") as f:
#     f.write(final_prompt)
    
# print(len(base_prompt),len(final_prompt))

In [7]:
base_prompt = "write a function that output the fibonacci sequence up to a given number"

In [8]:
prompt_agent = PromptAgent(base_prompt)

code_quality_principals = PromptReviewPrincipals([
    "Always write clean code",
    "Always write tests",
    "Always write documentation"
    ])
code_quality_reviewer = PromptReviewAgent("software engineer", code_quality_principals)

architecture_quality_principals = PromptReviewPrincipals([
    "Always design for scalability",
    "Always design for maintainability",
    "Always design for performance"
    ])
architecture_quality_reviewer = PromptReviewAgent("software architect", architecture_quality_principals)

writer_quality_principals = PromptReviewPrincipals([
    "Always write clear and concise instructions",
    "Always write with reference to specific context",
    "Always write with the end user (an LLM) in mind"
    ])
writer_quality_reviewer = PromptReviewAgent("writer", writer_quality_principals)

# instructor_quality_principals = PromptReviewPrincipals([
#     "Always provide clear and concise instructions",
#     "Always provide relevant and useful instructions"
#     ])
# instructor_quality_reviewer = PromptReviewAgent("instructor", instructor_quality_principals)

# context_quality_principals = PromptReviewPrincipals([
#     "Always provide context to the task",
#     "Always provide a description of the context",
#     "Always explain the importance of the context"
#     ])
# context_quality_reviewer = PromptReviewAgent("context", context_quality_principals)

# examplar_quality_principals = PromptReviewPrincipals([
#     "Always provide clear and concise examples",
#     "Always provide examples relevant to the task",
#     ])
# examplar_quality_reviewer = PromptReviewAgent("examplar", examplar_quality_principals)

# stylist_quality_principals = PromptReviewPrincipals([
#     "Always provide output in an appropriate format",
#     "Always provide instructions with an appropriate tone",
#     "Always use appropriate syntax and grammar"
#     ])
# stylist_quality_reviewer = PromptReviewAgent("stylist", stylist_quality_principals)

# constraint_quality_principals = PromptReviewPrincipals([
#     "Always define necessary constraints",
#     "Always explain why the constraints are necessary",
#     ])
# constraint_quality_reviewer = PromptReviewAgent("constraint", constraint_quality_principals)

sequential_prompt_review = SequentialPromptReview(prompt_agent)
sequential_prompt_review.register_approver(code_quality_reviewer)
sequential_prompt_review.register_approver(architecture_quality_reviewer)
sequential_prompt_review.register_approver(writer_quality_reviewer)
# sequential_prompt_review.register_approver(instructor_quality_reviewer)
# sequential_prompt_review.register_approver(context_quality_reviewer)
# sequential_prompt_review.register_approver(examplar_quality_reviewer)
# sequential_prompt_review.register_approver(stylist_quality_reviewer)
# sequential_prompt_review.register_approver(constraint_quality_reviewer)
final_prompt = await sequential_prompt_review.review()

# with open("prompt_before.txt", "w") as f:
#     f.write(base_prompt)

# with open("prompt_after.txt", "w") as f:
#     f.write(final_prompt)
    
print(len(base_prompt),len(final_prompt))

/Users/iwatson/Documents/Research Project/prompt-optimisation/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/iwatson/Documents/Research Project/prompt-optimisation/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [9]:
print(final_prompt)

**Updated Prompt:**

Write a function in Python that outputs the Fibonacci sequence up to a given number. Ensure the function adheres to the following guidelines:

1. **Clean Code:**
   - Use meaningful variable names.
   - Properly indent and structure the code.
   - Handle edge cases, such as non-integer inputs and negative numbers.
   - Avoid hard-coded values.

2. **Testing:**
   - Write unit tests to verify the function's behavior with various inputs, including edge cases and typical use cases.

3. **Documentation:**
   - Include docstrings or comments to explain the function's purpose, input parameters, and expected output.
   - Provide examples of how to use the function.

Here is an outline to get you started:

```python
def fibonacci_sequence(n):
    """
    Generate the Fibonacci sequence up to a given number n.

    Parameters:
    n (int): The upper bound for the Fibonacci sequence. Must be a non-negative integer.

    Returns:
    list: A list containing the Fibonacci sequ

In [ ]:
from openai import OpenAI
client = OpenAI()

def generate(prompt):
    completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "user", "content": prompt},
      ]
    )

    return completion.choices[0].message

base_response=generate(base_prompt)
final_response=generate(final_prompt)

with open("response_before.txt", "w") as f:
    f.write(base_response.content)
    
with open("response_after.txt", "w") as f:
    f.write(final_response.content)